##### 원본
[https://python.langchain.com/v0.2/docs/tutorials/rag/](https://python.langchain.com/v0.2/docs/tutorials/rag/)

# Q&A with RAG

## 개요
### LLM의 대표적인 문제 및 한계
>1. 할루시네이션(Hallucination)
>- 편향되거나 불충분한 학습데이터와 모델 과적화와 같은 이유로 정확하지 않거나 조작된 정보를 생성
>2. 높은 개발 및 훈련 비용
>- 서비스에 맞는 모델 훈련 및 개발에 엄청난 데이터/연산력과 함께 이를 위한 비용까지 상당히 높음
>3. 정보 보호
>- LLM 모델을 훈련 및 개발하는 과정에서 개인과 기업 데이터, 정보 유출이 일부 발생
>4. 편향과 공정성
>- 어느 한쪽으로 치우진 데이타를 학습했다면, 그것과 더 자주 연상시키는 문장을 새엉
>5. 투명성
>- 어떤 질문에 대해 특정 대답을 하는 경우 왜 그런 대답을 했는지 그 이유를 사용자에게 설명하는 능력이 부족
>6. 데이터 의존성
>- 모델이 특정 국가의 정보로만 학습을 할 경우 다른 지역의 문화적 맥락을 반영한 텍스트를 생성하는 데 한계
>7. 정보의 일반화
>- 너무 다양한 데이타를 학습했기 때문에 특정 산업에 특화된 질문을 할 경우 정밀한 답변을 얻지 못함
>8. 새로운 정보의 결여
>- 학습한 이후에 발생한 사건이나 정보에 대해서는 알지 못하고 과거의 정보를 기반으로 응답할 수 있음
>9. 기업 내 데이터 미활용
>- 각 기업에서만 가지고 있는 데이터가 포함되어 있지 않아 기업이 가진 데이타를 활용할 수 없음

이러한 문제를 해결할 수 있는 기술이 바로 RAG입니다.

### RAG란 무엇인가요?
LLM은 광범위한 주제에 대해 추론할 수 있지만 지식은 학습을 받은 특정 시점까지의 공개 데이터로 제한됩니다. 비공개 데이터나 모델 마감일 이후에 도입된 데이터에 대해 추론할 수 있는 AI 애플리케이션을 구축하려면 필요한 특정 정보로 모델에 대한 지식을 늘려야 합니다.

RAG(검색 증강 생성)는 LLM이 학습한 데이터 외에 외부 데이터베이스나 문서 집합에서 관련 정보를 검색한후, 적절한 정보를 가져와 모델 프롬프트에 삽입하여 새로운 텍스트를 생성함으로써 답변의 최신성과 정확도를 높여주는 도구이며 추가 데이타로 LLM 지식을 강화하는 기술입니다.

### 왜 RAG인가요?
- 기업이 원하는 맞춤형 LLM 모델을 만들수 있다.
- 비용적으로 저렴하고 외부 데이터를 지속적으로 새로이 가져올 수 있어 최신성 문제를 해결할 수 있다.
- '근거있는 생성'을 통해 LLM 태생적 문제인 할루시네이션 대비가 가능하다.
- 데이터 변동성에 유리하다.

## Q&A 아키텍처

우리는 위에서 개요한 대로 전형적인 RAG 애플리케이션을 만들 것입니다. 여기에는 두 가지 주요 구성 요소가 있읍니다:

- **인덱싱**: 소스에서 데이터를 수집하고 인덱싱하는 파이프라인입니다. _이 작업은 보통 오프라인에서 발생합니다._

- **검색 및 생성**: 실제 RAG 체인으로, 사용자 쿼리를 실행 시간에 받아 인덱스에서 관련 데이터를 검색한 다음, 가장 관련성이 높은 문서와 사용자 쿼리를 결합하여 LLM에 전달합니다.

RAW 데이터에서 답변을 받기까지의 전체 순서는 다음과 같습니다.

### 인덱싱

![](./images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

1. **로드**: 먼저 데이터를 로드해야 합니다. 이를 위해 [DocumentLoaders](https://python.langchain.com/docs/modules/data_connection/document_loaders/)를 사용할 것입니다.
2. **분할**: [Text splitters](https://python.langchain.com/docs/modules/data_connection/document_transformers/)는 큰 `Documents`를 더 작은 청크로 나눕니다. 이는 데이터를 인덱싱하고 모델에 전달하는 데 유용하며, 큰 청크는 검색하기 어렵고 모델의 유한한 컨텍스트 창에 맞지 않습니다.
3. **저장**: 나중에 검색할 수 있도록 분할을 저장하고 인덱싱할 장소가 필요합니다. 이는 종종 [VectorStore](https://python.langchain.com/docs/modules/data_connection/vectorstores/)와 [Embeddings](https://python.langchain.com/docs/modules/data_connection/text_embedding/) 모델을 사용하여 수행됩니다.

### 검색 및 생성

![](./images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

1. **검색**: 사용자 입력이 주어지면 [Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/)를 사용하여 저장소에서 관련 분할을 검색합니다.
2. **생성**: [ChatModel](https://python.langchain.com/docs/modules/model_io/chat/) / [LLM](https://python.langchain.com/docs/modules/model_io/llms/)은 질문과 검색된 데이터를 포함한 프롬프트를 사용하여 답변을 생성합니다


## 환경설정


### 의존성

여기에서는 OpenAI 챗 모델과 임베딩 및 Chroma 벡터 저장소를 사용하지만 여기에 사용된 모든 내용은 [ChatModel](https://python.langchain.com/v0.1/docs/modules/model_io/chat/) 또는 [LLM](https://python.langchain.com/v0.1/docs/modules/model_io/llms/), [임베딩](https://python.langchain.com/v0.1/docs/modules/data_connection/text_embedding/), [VectorStore](https://python.langchain.com/v0.1/docs/modules/data_connection/vectorstores/) 또는 [Retriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/)에서 작동합니다.


다음 패키지를 사용합니다.

In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

임베딩 모델에 대한 환경 변수 OPENAI_API_KEY를 설정해야합니다. 다음과 같이 .env 파일에서 로드할 수 있습니다.

In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

LangChain으로 구축한 애플리케이션은 여러 단계에 걸쳐 LLM 호출을 여러 번 사용하게 됩니다. 이러한 애플리케이션이 점점 더 복잡해짐에 따라, 체인이나 에이전트 내부에서 정확히 무슨 일이 일어나고 있는지 조사할 수 있는 능력이 매우 중요해집니다. 이를 위한 최선의 방법은 [LangSmith](https://smith.langchain.com)를 사용하는 것입니다.

LangSmith가 필수는 아니지만, 유용합니다. LangSmith가 애플리케이션을 추적하고 이해하는 데 어떻게 도움이 되는지 살펴보겠습니다. LangSmith는 애플리케이션이 복잡해짐에 따라 점점 더 유용해질 것입니다.

LangSmith를 사용하고 싶다면, 위의 링크에서 가입한 후, 로깅 추적을 시작하기 위해 아래처럼 환경 변수를 설정하거나 .env 파일에서 로드할 수 있습니다.

In [2]:
import os

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "basic"

# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
os.environ["LANGCHAIN_TRACING_V2"] = "true"

## 시연

네이버 뉴스기사의 내용에 대해 질문할 수 있는 **뉴스기사 QA 앱** 을 구축할 것입니다.

무슨 일이 일어나고 있는지 실제로 이해하기 위해 단계별로 살펴보겠습니다.

### 1. 인덱싱: 로드

먼저 뉴스 콘텐츠를 로드해야 합니다. 이를 위해 소스에서 데이터를 로드하고 [문서](https://api.python.langchain.com/en/latest/documents/langchain_core.documents.base.Document.html) 목록을 반환하는 객체인 [DocumentLoaders](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/)를 사용할 수 있습니다. 문서는 paqge_content(str)와 메타데이타(dict)가 포함된 객체입니다.

이를 위해 urllib를 사용하여 웹 URL에서 HTML을 로드하고 BeautifulSoup을 사용하여 이를 텍스트로 구문 분석하는 WebBaseLoader를 사용합니다. bs_kwargs를 통해 BeautifulSoup 파서에 매개변수를 전달하여 HTML 텍스트 구문 분석을 사용자 정의할 수 있습니다. ([BeautifulSoup 문서](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup) 참조). 이 경우 "newsct_article _article_body", "media_end_head_title" class가 있는 HTML 태그만 관련되므로 다른 모든 태그는 제거됩니다. 

In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# 뉴스기사 내용을 로드하고, 청크로 나누고, 인덱싱합니다.
url = "https://n.news.naver.com/article/437/0000378416"
url = "https://n.news.naver.com/article/031/0000838386?cds=news_edit"
loader = WebBaseLoader(
    web_paths=(url,),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body","media_end_head_title"]},
        )
    ),
)

docs = loader.load()
print(f"문서 내용의 길이: {len(docs[0].page_content)}")
docs

문서 내용의 길이: 1846


[Document(page_content='\n"치킨값 3만원 돼야" 강조했던 BBQ…결국 \'가장 비싼 후라이드\'\n\n\nBBQ, 빅3 치킨 프랜차이즈 중 먼저 가격인상 카드 뽑아들어원·부재료 등 가격 앙등 못 이기고 황금올리브유 2년만에 3000원↑2017년 BBQ 가격 인상 땐 공정위 조사 나서자 결국 철회하기도국내 치킨 프랜차이즈 주요 3사(BHC·BBQ·교촌) 중 BBQ(비비큐)치킨이 가장 먼저 가격 인상 카드를 뽑아들었다. 최근 원·부재료 가격 상승과 최저임금, 임차료 등이 오른 것이 가격 인상 요인으로 꼽힌다.\n\n\n\n황금 올리브 치킨. [사진=BBQ]22일 BBQ는 오는 23일부터 인기 품목 \'황금올리브치킨 후라이드\' 가격을 기존\xa02만원에서\xa02만3000원으로 인상한다고 밝혔다. 또 \'자메이카 통다리구이\'도 기존 2만1500원에서\xa02만4000원으로 가격을 올린다.BBQ 측은 총 110개 판매 제품 중 23개 제품의 소비자 권장 판매 가격을 인상하고, 56개 제품 가격은 동결, 31개 제품에 대해서는 증량을 통해 가격인하 효과를 줬다고 설명했다.하지만 BBQ가 정부의 물가안정 기조에도 불구하고, 가격 인상을 선언하면서 소비자들은 물론, 정부의 곱지 않은 시선을 받을 수 있다는 지적이 나온다.BBQ가 가격 인상을 공식화하자 소비자들은 "치킨 가격 3만원이 현실화 되는 것 아니냐"는 우려의 목소리를 내고 있다. 윤홍근 BBQ 회장은 2022년 3월 한 라디오 방송에 출연해 "치킨 가격이 3만원은 돼야 한다"고 밝히며 치킨 가격 적정성 논란을 불러 일으켰었다. 당시 BBQ \'황금올리브치킨 후라이드\' 가격은 2만원이었으며, BBQ는 2년만에 3000원을 올리면서 \'3만원 치킨\'에 더욱 가까워진 상황이다.BBQ는 지난해 치킨유를 100% 올리브유에서, 올리브유와 해바라기유를 절반씩 섞은 \'혼합유\'로 치킨유를 바꿨다. 국제 올리브유 가격이 크게 치솟으면서 가격 인상 부담을 줄이기 위한 조치였다. 당시 가격 인상 대신 

### 2. 인덱싱: 분할

로드된 문서의 길이가 길 경우 많은 모델의 컨텍스트 window에 맞지 않습니다. 컨텍스트 window 에 전체 게시물을 맞출 수 있는 모델의 경우에도 모델은 매우 긴 입력에서 정보를 찾는 데 어려움을 겪을 수 있습니다.

이를 처리하기 위해 문서를 삽입 및 벡터 저장을 위한 chunk로 분할합니다. 이는 런타임 시 뉴스 게시물의 가장 관련성이 높은 부분만 검색하는 데 도움이 됩니다.

이 경우 문서를 1000자의 chunk로 분할하고 chunk 사이에 200자가 겹치도록 하겠습니다. 중복은 진술과 관련된 중요한 맥락에서 진술을 분리할 가능성을 완화하는 데 도움이 됩니다.우리는 각 chunk가 적절한 크기가 될 때까지 새 줄과 같은 공통 구분 기호를 사용하여 문서를 재귀적으로 분할하는 [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/)를 사용합니다.이는 일반적인 텍스트 사용 사례에 권장되는 텍스트 분할기입니다.

초기 Document 내에서 각 분할 Document가 시작되는 문자 인덱스가 메타 데이터 속성 'start_index'로 유지되도록 add_start_index=True를 설정했습니다.


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000, chunk_overlap=100, add_start_index=True
)

all_splits = text_splitter.split_documents(docs)
len(all_splits)
len(all_splits[0].page_content)
all_splits[0].metadata

{'source': 'https://n.news.naver.com/article/031/0000838386?cds=news_edit',
 'start_index': 1}

### 3. 인덱싱 : 저장

이제 런타임에 검색할 수 있도록 텍스트 정크를 인덱싱해야 합니다. 이를 수항해는 가장 일반적인 방법은 분할된 각 문서의 내용을 포함하고 이러한 포함을 벡터 데이터베이스(또는 벡터 저장소)에 삽입하는 것입니다. 분할을 검색하려는 경우 텍스트 검색 쿼리를 가져와 이를 포함하고 일종의 '유사성' 검색을 수행하여 쿼리 포함과 가장 유사한 임베딩이 있는 저장된 분할을 식별합니다. 가장 간단한 유사성 측정은 코사인 유사성입니다. 각 임베딩 쌍(고차원 벡터)사의 각도의 코사인을 측정합니다.

[Chroma](https://python.langchain.com/v0.1/docs/integrations/vectorstores/chroma/) 벡터 저장소와 [OpenAIEmbeddings](https://python.langchain.com/v0.1/docs/integrations/text_embedding/openai/) 모델을 사용하여 단일 명령에 모든 문서 분할을 포함하고 저장할 수 있습니다.


In [6]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# 벡터스토어를 생성합니다.
vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())


### 4. 검색과 생성: 검색하기

이제 실제 애플리케이션 로직을 작성해 보겠습닏. 우리는 사용자 질문을 받아 해당 질문과 관련된 문서를 검색하고 검색된 문서와 초기 질문을 모델에 전달하고 답변을 반환하는 간단한 애플리케이션을 만들고 싶습니다.

먼저 문서 검색 논리를 정의해야 합니다. LangChain은 문자열 쿼리가 주어지면 관련 문서를 반활할 수 있는 인덱스를 래핑하는 [Retriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/) 인터페이스를 정의합니다.

가장 일반적인 유형의 Retriver는 벡터 저장소의 유사성 검색 기능을 사용하여 검색을 용이하게 하는 [VectorStoreRetriever](https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/)입니다. VectorStore.as_retriever()를 사용하면 모든 VectorStore를 Retriever로 쉽게 전환할 수 있습니다.

In [8]:
# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
# search_type: 매개변수에 검색 알고리즘을 지정할 수 있습니다. (similarity, mmr, similarity_score_threshold등의 옵션 지정)
# search_kwargs: k - 매개변수로 찾을 문서의 개수를 지정합니다. {"k": 2}로 지정한다면
# 2개의 유사도 높은 문서만 선택하겠다라는 의미입니다. 기본값은 1입니다.
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 2})
retrieved_docs = retriever.invoke("치킨 가격이 어느정도 인상되었나요?")
len(retrieved_docs)
# print(retrieved_docs[0].page_content)

2

### 5. 검색과 생성: 생성하기

질문을 받고, 관련 문서를 검색하고, 프롬프트를 구성하고, 이를 모델에 전달하고, 출력을 구문 분석하는 체인에 이 모든 것을 함께 넣어 보겠습니다.

여기서는 gtp-3.5-turbo OpenAI 챗 모델을 사용할 것이지만 LangChain LLM 또는 ChatModel을 대체할 수 있습니다.

In [9]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [10]:
print(prompt.messages[0].prompt.template)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:


우리는 [LCEL Runnable](https://python.langchain.com/v0.1/docs/expression_language/) 프로토콜을 사용하여 체인을 정의합니다. - 투명한 방식으로 구성 요소와 기능을 함께 연결합니다. - LangSmith에서 자동으로 체인을 추적합니다. - 스트리밍, 비동기 및 일괄 호출을 즉시 사용할 수 있습니다.

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=4096
)

def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

이 함수는 `rag_chain` 객체의 `invoke` 메서드를 사용하여 사용자의 질문을 처리합니다.


In [13]:
question = "기사에서 언급된 치킨을 알려주세요."
response = rag_chain.invoke(question) # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

# 결과 출력
print(f"URL: {url}")
print(f"문서의 수: {len(docs)}")
print("===" * 20)
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

URL: https://n.news.naver.com/article/031/0000838386?cds=news_edit
문서의 수: 1
[HUMAN]
기사에서 언급된 치킨을 알려주세요.

[AI]
BBQ치킨


In [14]:
print(rag_chain.invoke(
    "치킨 가격을 인상하는 업체는 어디인가요?"
)) # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

BBQ치킨이 치킨 가격을 인상했습니다. BBQ치킨은 원·부재료 가격 상승과 최저임금, 임대료 등의 영향으로 가격을 조정했습니다. BBQ치킨은 공정거래위원회의 가맹업법 위반 혐의 조사를 받았으며, 교촌치킨도 가격 인상 계획을 철회했습니다.


In [15]:
print(
    rag_chain.invoke("치킨 가격을 인상하는 원인이 무엇인지 bullet points 형식으로 작성해 주세요.")
)  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

- BBQ치킨은 가맹업법 위반 혐의 조사로 인해 가격 인상을 결정하지 못했음
- 교촌치킨은 BBQ에 대한 조사가 시작되자 가격 인상 계획을 철회함
- 굽네치킨과 푸라닭 치킨도 가격을 올리는 등 업계 전반적으로 가격이 상승함


In [16]:
print(rag_chain.invoke(
    "배달앱 수수료는 얼마인가요?"
))  # 문서에 대한 질의를 입력하고, 답변을 출력합니다.

배달앱 수수료에 대한 구체적인 정보는 제공되지 않았습니다.


In [17]:
# clean up
vectorstore.delete_collection()